In [1]:
dataset_id = "college_notebook_staging"

In [2]:
!bq --location=US mk --dataset {dataset_id}

Dataset 'cs327e-fa2019:college_notebook_staging' successfully created.


## Loading CSV files from Cloud Storage

In [3]:
!bq --location=US load --autodetect --skip_leading_rows=1 \
--source_format=CSV {dataset_id}.Current_Student \
'gs://college_dataset/raw/Current_Students.csv'

Waiting on bqjob_r6d1ccc823231ea4d_0000016ea37735f2_1 ... (8s) Current status: DONE   


In [4]:
%%bigquery 
select count(*) from college_notebook_staging.Current_Student

,f0_
0,11


In [5]:
!bq --location=US load --autodetect --skip_leading_rows=1 \
--source_format=CSV {dataset_id}.New_Student \
'gs://college_dataset/raw/New_Students.csv'

Waiting on bqjob_r4325982ded67a69f_0000016ea3776730_1 ... (15s) Current status: DONE   


In [6]:
%%bigquery 
select count(*) from college_notebook_staging.New_Student

,f0_
0,7


In [7]:
!bq --location=US load --autodetect --skip_leading_rows=1 \
--source_format=CSV {dataset_id}.Class \
'gs://college_dataset/raw/Classes.csv'

Waiting on bqjob_r209ce4cb82403402_0000016ea377b0d4_1 ... (1s) Current status: DONE   


In [8]:
%%bigquery 
select count(*) from college_notebook_staging.Class

,f0_
0,12


## Creating Modeled Tables 

In [9]:
modeled_dataset_id = "college_notebook_modeled"

In [10]:
!bq --location=US mk --dataset {modeled_dataset_id}

Dataset 'cs327e-fa2019:college_notebook_modeled' successfully created.


In [11]:
%%bigquery
create or replace table college_notebook_modeled.Student as
select distinct sid, fname, lname, dob
from college_notebook_staging.Current_Student
union distinct
select distinct sid, fname, lname, cast(dob as string) as dob
from college_notebook_staging.New_Student

""


In [12]:
%%bigquery
select count(*) from college_notebook_modeled.Student

,f0_
0,15


In [13]:
%%bigquery
create or replace table college_notebook_modeled.Teacher as
select distinct tid, instructor, dept
from college_notebook_staging.Class

""


In [14]:
%%bigquery
select count(*) from college_notebook_modeled.Teacher

,f0_
0,12


In [15]:
%%bigquery
select * from college_notebook_modeled.Teacher limit 10

,tid,instructor,dept
0,cannata,"Cannata, Philip",CS
1,cannata,PHILIP CANNATA,CS
2,mitra,"Mitra, Shyamal",CS
3,None,Prof. Alison Norman,CS
4,None,Prof. Glenn Downing,CS
5,mueller,"MUELLER, PETER",Math
6,koch,"Koch, Hans",Math
7,neeman,JOE NEEMAN,Mathematics
8,tran,"Tran, Ngoc",Mathematics
9,bulko,bill bulko,Computer Science


In [ ]:
%%bigquery
create or replace table college_notebook_modeled.Takes as
select distinct sid, cno, grade
from college_notebook_staging.Current_Student
where sid is not null
and cno is not null

In [ ]:
%%bigquery
select * from college_notebook_modeled.Takes limit 10

In [ ]:
%%bigquery 
create table college_notebook_modeled.Teaches as
select distinct tid, cno
from college_notebook_staging.Class
where tid is not null
and cno is not null

In [ ]:
%%bigquery
select * from college_notebook_modeled.Teaches limit 10